In [1]:
# the model is in ../models/Llama-3.2-1B-Instruct
# load the llama model at try to do the chat

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "meta-llama/Llama-3.2-1B-Instruct"
model_path = "../models/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path= model_path,
    device_map="auto",
    torch_dtype=torch.float16,
)
model.to(device)
model.eval()

D:\applications\anaconda\envs\hku_nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
      (1): LlamaDecoderLayer(

In [1]:
import json
product = "Furniture-Bookcases"
# read the json file at ../data_analysis/analysis_result_{}/analysis_result_{}.json
with open(f"../data_analysis/analysis_result_{product}/analysis_result_{product}.json", "r") as f:
    analysis_result = json.load(f)
print(analysis_result.keys())

# segment_distribution_text
segment_distribution = analysis_result["segment_distribution"]
sorted_segment_distribution = sorted(segment_distribution.items(), key=lambda item: item[1][0], reverse=True)
segment_distribution_text = "The highest segment for {} is {}, which accounts for {:.2f}%.".format(product, sorted_segment_distribution[0][0], sorted_segment_distribution[0][1][1]*100)
print(segment_distribution_text)

# segment_quantity_analysis
segment_quantity_analysis = analysis_result["segment_quantity"]
sorted_segment_quantity_analysis = sorted(segment_quantity_analysis.items(), key=lambda item: item[1], reverse=True)
segment_quantity_analysis_text = "The highest segment quantity is {}, which bought average {} {} each time.".format(sorted_segment_quantity_analysis[0][0], sorted_segment_quantity_analysis[0][1], product)
print(segment_quantity_analysis_text)

# segment_unit_price_analysis
segment_unit_price_analysis = analysis_result["segment_unit_price"]
sorted_segment_unit_price_analysis = sorted(segment_unit_price_analysis.items(), key=lambda item: item[1], reverse=True)
segment_unit_price_analysis_text = "The highest segment unit price is {}, which paid {} for each {}.".format(sorted_segment_unit_price_analysis[0][0], sorted_segment_unit_price_analysis[0][1], product)
print(segment_unit_price_analysis_text)

# segment_profit_analysis
segment_profit_analysis = analysis_result["segment_profit"]
sorted_segment_profit_analysis = sorted(segment_profit_analysis.items(), key=lambda item: item[1], reverse=True)
segment_profit_analysis_text = "The highest segment profit is {}, which generated {} profit.".format(sorted_segment_profit_analysis[0][0], sorted_segment_profit_analysis[0][1])
print(segment_profit_analysis_text)

dict_keys(['segment_distribution', 'segment_quantity', 'segment_unit_price', 'segment_profit', 'segment_discount', 'region_distribution', 'top_10_states', 'top_10_cities'])
The highest segment for Furniture-Bookcases is Consumer, which accounts for 58.00%.
The highest segment quantity is Corporate, which bought average 4.16 Furniture-Bookcases each time.
The highest segment unit price is Consumer, which paid 138.61 for each Furniture-Bookcases.
The highest segment profit is Home Office, which generated 3.21 profit.


In [2]:
known_facts = str({
    "product": product,
    "most_segment_customers": segment_distribution_text,
    "most_quantity_customers": segment_quantity_analysis_text,
    "highest_unit_price_analysis": segment_unit_price_analysis_text,
    "highest_profit_analysis": segment_profit_analysis_text,
})

In [57]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
input_text = "For product {},known facts are: {},write a precise analysis less than 500 words".format(product, known_facts)
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
output = model.generate(**input_ids, max_length=2048, do_sample=True, temperature=0.1)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


For product Furniture-Bookcases,known facts are: {'product': 'Furniture-Bookcases','most_segment_customers': 'The highest segment for Furniture-Bookcases is Consumer, which accounts for 58.00%.','most_quantity_customers': 'The highest segment quantity is Corporate, which bought average 4.16 Furniture-Bookcases each time.', 'highest_unit_price_analysis': 'The highest segment unit price is Consumer, which paid 138.61 for each Furniture-Bookcases.', 'highest_profit_analysis': 'The highest segment profit is Home Office, which generated 3.21 profit.'},write a precise analysis less than 500 words.

## Step 1: Identify the key facts about Furniture-Bookcases
The given facts about Furniture-Bookcases include the product name, the most segment customers, the most quantity customers, the highest unit price analysis, and the highest profit analysis.

## Step 2: Analyze the most segment customers
The most segment customers for Furniture-Bookcases is Consumer, which accounts for 58.00%. This sugges

In [14]:
eval(known_facts)

{'product': 'Furniture-Bookcases',
 'most_segment_customers': 'The highest segment for Furniture-Bookcases is Consumer, which accounts for 58.00%.',
 'most_quantity_customers': 'The highest segment quantity is Corporate, which bought average 4.16 Furniture-Bookcases each time.',
 'highest_unit_price_analysis': 'The highest segment unit price is Consumer, which paid 138.61 for each Furniture-Bookcases.',
 'highest_profit_analysis': 'The highest segment profit is Home Office, which generated 3.21 profit.'}

In [11]:
# Another choice: using api from zhipu ai
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="06f59aab3f61a4e8ba0ef45b663fc204.ipiw6sxXKMQNoBY6")  # 请填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4-plus",  # 请填写您要调用的模型名称
    messages=[
        {"role": "user", "content": "As a Sales Assistant to help the company, please analysis and generate sales suggestions and advise for the product {}, with less than 200 words.".format(product)},
        {"role": "assistant", "content": "Of course, I can help you with that. Could you provide me with some information about the product?"},
        {"role": "user", "content": "Here are the facts: {}".format(known_facts)},
    ],
)
print(response.choices[0].message)

CompletionMessage(content='Based on the provided data, here are targeted sales suggestions for Furniture-Bookcases:\n\n1. **Focus on Consumer Segment**: Since consumers account for 58% of sales, tailor marketing efforts to highlight bookcases as stylish home accessories. Emphasize design, durability, and space-saving features in advertising.\n\n2. **Corporate Bundle Deals**: Offer bulk discounts or package deals to Corporate clients who average 4.16 units per purchase. This could incentivize larger orders, boosting overall sales volume.\n\n3. **Pricing Strategy for Consumers**: Capitalize on the high unit price ($138.61) consumers are willing to pay by introducing premium models or customization options, justifying the higher cost.\n\n4. **Enhance Home Office Appeal**: Despite lower sales, the Home Office segment yields the highest profit per unit ($3.21). Develop targeted campaigns showcasing bookcases as essential home office organizers, potentially increasing both sales and profit m